In [597]:
import pandas as pd
import numpy as np

Необходимо привести данные к виду:
|Имя дисциплины|Посещаемость лекций 1 пром|Посещаемость лекций 2 пром|Посещаемость лекций 3 пром|Посещаемость практик 1 пром|Посещаемость практик 2 пром|Посещаемость практик 3 пром|КТ 1|КТ 2|Экзамен|Балл ктр|Балл лаб|
|-|-|-|-|-|-|-|-|-|-|-|-|
|Имя-семестр|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|1|1|

# Реализация

Берем таблицу attendance-students

In [598]:
attendance_students = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-students.csv').rename(columns={'id':'attendance_id', 'contingent_id':'con_id'})

attendance_students.head(5)

,attendance_id,con_id
0,3648,136592
1,3657,140543
2,3677,140322
3,3704,144453
4,3706,143684


Берем таблицу ocenka-students

In [599]:
ocenka_students = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-students.csv').rename(columns={'id':'ocenka_id', 'study_id':'con_id'})

ocenka_students.head(5)

,ocenka_id,con_id,enrollment_at,deleted_at,group_id
0,11162,156272,2018-01-11,2019-02-08 01:00:40 +0700,374
1,11191,140322,2018-02-28,2018-07-05 01:00:08 +0700,157
2,11202,147770,2018-03-02,2018-05-30 01:00:09 +0700,127
3,11246,154682,2018-03-09,2021-09-01 18:08:52 +0700,413
4,11269,154099,2018-04-04,2018-12-26 01:01:15 +0700,363


Добавляем эти таблицы в мастер-таблицу

In [600]:
master_table = attendance_students.set_index('con_id').join(ocenka_students.set_index('con_id')).dropna()

master_table.head(5)

,attendance_id,ocenka_id,enrollment_at,deleted_at,group_id
con_id,,,,,
136592,3648,11287,2018-04-13,2018-07-05 01:00:08 +0700,188
140543,3657,13537,2018-09-21,2019-06-19 01:00:09 +0700,136
140322,3677,11191,2018-02-28,2018-07-05 01:00:08 +0700,157
144453,3704,13523,2018-09-15,2019-07-14 01:00:09 +0700,28
143684,3706,13721,2018-09-27,2019-07-14 01:00:09 +0700,28


Удаляем из таблицы все кроме id

In [601]:
master_table.drop(columns=['enrollment_at', 'deleted_at'], axis=1, inplace=True)

master_table.head(5)

,attendance_id,ocenka_id,group_id
con_id,,,
136592,3648,11287,188
140543,3657,13537,136
140322,3677,11191,157
144453,3704,13523,28
143684,3706,13721,28


Добавляем в таблицу группы и удаляем все, кроме id студентов и групп.

В дальнейшем в принципе будет удаляться все, кроме интересующей информации, даже если придется её потом снова добавлять

In [602]:
ocenka_groups = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-groups.csv').rename(columns={'id':'group_id'})

ocenka_groups.head(5)

,group_id,number,edu_program_full_title
0,18,435-3,"09.03.01, Информатика и вычислительная техника..."
1,28,585-2,"09.03.01, Информатика и вычислительная техника..."
2,93,435-1,"09.03.01, Информатика и вычислительная техника..."
3,127,586-2,"09.03.01, Информатика и вычислительная техника..."
4,136,585-1,"09.03.01, Информатика и вычислительная техника..."


In [603]:
master_table = master_table.join(ocenka_groups.rename(columns={'number':'group_num'}).set_index('group_id'), on='group_id').dropna()

master_table.drop(['group_id', 'edu_program_full_title'], axis=1, inplace=True)

master_table.head(5)

,attendance_id,ocenka_id,group_num
con_id,,,
136592,3648,11287,584-2
140543,3657,13537,585-1
140322,3677,11191,584-1
144453,3704,13523,585-2
143684,3706,13721,585-2


Получим id групп со всех баз

In [604]:
attendance_groups = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-groups.csv').rename(columns={'id':'group_id'})

attendance_groups.head(5)

,group_id,number
0,233,584-1
1,234,584-2
2,341,435-1
3,343,435-3
4,353,585-1


In [605]:
timetable_groups = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-groups.csv').rename(columns ={'id':'group_id'})

timetable_groups.head(5)

,group_id,number,course_id
0,6816,438-3,1140
1,7111,438-3,1218
2,8357,438-3,1333
3,8358,438-3,1417
4,9061,438-3,1526


In [606]:
master_table = master_table.join(ocenka_groups[['group_id', 'number']].rename(columns={'number':'group_num', 'group_id':'group_id_ocenka'}).set_index('group_num'), on='group_num').dropna()
master_table = master_table.join(attendance_groups[['group_id', 'number']].rename(columns={'number':'group_num', 'group_id':'group_id_attendance'}).set_index('group_num'), on='group_num').dropna()
master_table = master_table.join(timetable_groups[['group_id', 'number']].rename(columns={'number':'group_num', 'group_id':'group_id_timetable'}).set_index('group_num'), on='group_num').dropna()

master_table = master_table.reset_index()

master_table.head(10)

,con_id,attendance_id,ocenka_id,group_num,group_id_ocenka,group_id_attendance,group_id_timetable
0,157460,11850,12100,438-3,482,623,6816.0
1,157460,11850,12100,438-3,482,623,7111.0
2,157460,11850,12100,438-3,482,623,8357.0
3,157460,11850,12100,438-3,482,623,8358.0
4,157460,11850,12100,438-3,482,623,9061.0
5,157460,11850,12100,438-3,482,623,9421.0
6,157460,11850,12100,438-3,482,623,10172.0
7,157460,11850,12100,438-3,482,623,10543.0
8,157465,11851,12101,438-3,482,623,6816.0
9,157465,11851,12101,438-3,482,623,7111.0


Добавляем данные о семестре

In [607]:
timetable_courses = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-courses.csv').rename(columns={'id':'course_id'})

timetable_courses.head(5)

,course_id,faculty_id,number
0,1140,194,1
1,1141,194,2
2,1143,194,4
3,1146,195,1
4,1147,195,2


In [608]:
timetable_faculties = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-faculties.csv').rename(columns={'id':'faculty_id'})

timetable_faculties.head(5)

,faculty_id,name,abbr,semester_id
0,194,Факультет систем управления,ФСУ,17
1,195,Факультет вычислительных систем,ФВС,17
2,207,Факультет систем управления,ФСУ,18
3,208,Факультет вычислительных систем,ФВС,18
4,225,Факультет вычислительных систем,ФВС,19


In [609]:
buffer_table = timetable_courses[['course_id', 'faculty_id']]

buffer_table.head(5)

,course_id,faculty_id
0,1140,194
1,1141,194
2,1143,194
3,1146,195
4,1147,195


In [610]:
buffer_table = buffer_table.join(timetable_faculties[['faculty_id', 'semester_id']].set_index('faculty_id'), on='faculty_id').dropna()

buffer_table.head(5)

,course_id,faculty_id,semester_id
0,1140,194,17
1,1141,194,17
2,1143,194,17
3,1146,195,17
4,1147,195,17


In [611]:
buffer_table = buffer_table.set_index(['course_id', 'faculty_id']).sub(buffer_table['semester_id'].min()).reset_index()[['course_id', 'semester_id']]

buffer_table.head(10)

,course_id,semester_id
0,1140,0
1,1141,0
2,1143,0
3,1146,0
4,1147,0
5,1148,0
6,1149,0
7,1218,1
8,1219,1
9,1221,1


In [612]:
master_table = master_table.join(timetable_groups[['group_id', 'course_id']].rename(columns={'group_id':'group_id_timetable'}).set_index('group_id_timetable'), on='group_id_timetable').dropna()

master_table.head(10)

,con_id,attendance_id,ocenka_id,group_num,group_id_ocenka,group_id_attendance,group_id_timetable,course_id
0,157460,11850,12100,438-3,482,623,6816.0,1140
1,157460,11850,12100,438-3,482,623,7111.0,1218
2,157460,11850,12100,438-3,482,623,8357.0,1333
3,157460,11850,12100,438-3,482,623,8358.0,1417
4,157460,11850,12100,438-3,482,623,9061.0,1526
5,157460,11850,12100,438-3,482,623,9421.0,1610
6,157460,11850,12100,438-3,482,623,10172.0,1707
7,157460,11850,12100,438-3,482,623,10543.0,1791
8,157465,11851,12101,438-3,482,623,6816.0,1140
9,157465,11851,12101,438-3,482,623,7111.0,1218


In [613]:
master_table = master_table.join(buffer_table.set_index('course_id'), on='course_id').dropna()

master_table.head(5)

,con_id,attendance_id,ocenka_id,group_num,group_id_ocenka,group_id_attendance,group_id_timetable,course_id,semester_id
0,157460,11850,12100,438-3,482,623,6816.0,1140,0
1,157460,11850,12100,438-3,482,623,7111.0,1218,1
2,157460,11850,12100,438-3,482,623,8357.0,1333,2
3,157460,11850,12100,438-3,482,623,8358.0,1417,3
4,157460,11850,12100,438-3,482,623,9061.0,1526,4


In [614]:
master_table.drop(['course_id'], axis=1,inplace=True)
master_table = master_table.rename(columns={'semester_id':'semester'})

master_table.head(5)

,con_id,attendance_id,ocenka_id,group_num,group_id_ocenka,group_id_attendance,group_id_timetable,semester
0,157460,11850,12100,438-3,482,623,6816.0,0
1,157460,11850,12100,438-3,482,623,7111.0,1
2,157460,11850,12100,438-3,482,623,8357.0,2
3,157460,11850,12100,438-3,482,623,8358.0,3
4,157460,11850,12100,438-3,482,623,9061.0,4


Находим все дисциплины группы

In [615]:
timetable_educations = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-educations.csv').rename(columns={'id':'edu_id'})

timetable_educations.head(5)

,edu_id,group_id,discipline_id
0,8582,1285,3177
1,8583,1285,3178
2,8584,1285,3179
3,8585,1285,3215
4,8586,1285,3216


In [616]:
timetable_disciplines = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-disciplines.csv').rename(columns={'id':'discipline_id'})

timetable_disciplines.head(5)

,discipline_id,semester_id,course_id,name
0,19026,13,848,"Вычислительные машины, системы и сети"
1,19027,13,848,Метрология и измерительная техника
2,19028,13,848,Математические основы теории систем
3,19029,13,848,Информационные сети и телекоммуникации
4,19030,13,848,Элективные дисциплины по физической культуре и...


In [617]:
buffer_table = timetable_educations

buffer_table = buffer_table.join(timetable_disciplines[['discipline_id', 'name']].rename(columns={'name':'names'}).set_index('discipline_id'), on='discipline_id').dropna()

buffer_table.head(5)

,edu_id,group_id,discipline_id,names
282,45725,5001,19146,Сети и телекоммуникации
283,45726,5001,19147,"Теория вероятностей, математическая статистика..."
284,45727,5001,19148,Методы оптимальных решений
285,45728,5001,19030,Элективные дисциплины по физической культуре и...
286,45729,5001,19149,Теоретические основы механики / Механика


In [618]:
master_table = master_table.join(buffer_table[['group_id', 'discipline_id', 'names']].rename(columns={'discipline_id':'discipline_id_timetable', 'names':'dicipline_name_timetable'}).set_index('group_id'), on='group_id_timetable').dropna().reset_index()

master_table.drop(['index'], axis=1,inplace=True)

master_table.head(5)

,con_id,attendance_id,ocenka_id,group_num,group_id_ocenka,group_id_attendance,group_id_timetable,semester,discipline_id_timetable,dicipline_name_timetable
0,157460,11850,12100,438-3,482,623,6816.0,0,23733.0,История
1,157460,11850,12100,438-3,482,623,6816.0,0,23729.0,Иностранный язык
2,157460,11850,12100,438-3,482,623,6816.0,0,23734.0,Физическая культура и спорт
3,157460,11850,12100,438-3,482,623,6816.0,0,23766.0,Математика
4,157460,11850,12100,438-3,482,623,6816.0,0,23741.0,Русский язык и культура речи


Считаем посещаемость

In [619]:
attendance_lessons = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-lessons.csv")

attendance_lessons.head(5)

,id,group_id,discipline_id,date_on,kind,order_number,timetable_id
0,650,233,20,2014-09-02,lecture,1,100206
1,651,233,251,2014-09-02,practice,2,100214
2,698,234,20,2014-09-02,lecture,1,100206
3,699,234,214,2014-09-02,practice,2,100215
4,1256,233,214,2014-09-03,lecture,1,100230


In [620]:
attendance_disciplines = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-disciplines.csv")

attendance_disciplines.head(5)

,id,title
0,2,Теория вероятностей и математическая статистика
1,4,Правоведение
2,11,Организация и планирование производства
3,13,Информационные технологии
4,14,Социология


In [621]:
attendance_presences = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-presences.csv")

attendance_presences['state'].fillna('wasnt', inplace=True)

attendance_presences.head(5)

,id,student_id,lesson_id,state
0,10408,3704,698,wasnt
1,10410,3706,698,wasnt
2,10413,3709,698,wasnt
3,10422,3718,698,wasnt
4,10430,3704,699,wasnt


In [622]:
buffer_table = attendance_lessons[['id', 'group_id', 'discipline_id', 'date_on', 'kind']].loc[((attendance_lessons['kind'] == 'lecture') |(attendance_lessons['kind'] == 'practice'))]
buffer_table.head(5)

,id,group_id,discipline_id,date_on,kind
0,650,233,20,2014-09-02,lecture
1,651,233,251,2014-09-02,practice
2,698,234,20,2014-09-02,lecture
3,699,234,214,2014-09-02,practice
4,1256,233,214,2014-09-03,lecture


In [623]:
buffer_table = buffer_table.join(attendance_presences[['student_id', 'lesson_id', 'state']].set_index('lesson_id'), on='id').rename(columns={'date_on':'date'}).dropna().reset_index()

buffer_table.drop(['index', 'id'], axis=1, inplace=True)

buffer_table.head()

,group_id,discipline_id,date,kind,student_id,state
0,234,20,2014-09-02,lecture,3704.0,wasnt
1,234,20,2014-09-02,lecture,3706.0,wasnt
2,234,20,2014-09-02,lecture,3709.0,wasnt
3,234,20,2014-09-02,lecture,3718.0,wasnt
4,234,214,2014-09-02,practice,3704.0,wasnt


In [624]:
counter = 0
for i in range(2018,2030):
    buffer_table.loc[(buffer_table['date'] >= f'{i}-09-01') & (buffer_table['date'] < f'{i}-10-16'), 'period'] = 1
    buffer_table.loc[(buffer_table['date'] >= f'{i}-10-16') & (buffer_table['date'] < f'{i}-12-04'), 'period'] = 2
    buffer_table.loc[(buffer_table['date'] >= f'{i}-12-04') & (buffer_table['date'] < f'{i+1}-01-15'), 'period'] = 3
    
    buffer_table.loc[(buffer_table['date'] >= f'{i}-09-01') & (buffer_table['date'] <= f'{i+1}-01-15'), 'semester'] = counter
    counter+=1
    
    
    buffer_table.loc[(buffer_table['date'] >= f'{i+1}-02-03') & (buffer_table['date'] < f'{i+1}-03-15'), 'period'] = 1
    buffer_table.loc[(buffer_table['date'] >= f'{i+1}-03-15') & (buffer_table['date'] < f'{i+1}-04-15'), 'period'] = 2
    buffer_table.loc[(buffer_table['date'] >= f'{i+1}-04-15') & (buffer_table['date'] <= f'{i+1}-06-30'), 'period'] = 3
    
    buffer_table.loc[(buffer_table['date'] >= f'{i+1}-02-03') & (buffer_table['date'] <= f'{i+1}-06-30'), 'semester'] = counter
    counter+=1

buffer_table.dropna(inplace=True)

buffer_table.drop('date', axis=1, inplace=True)

buffer_table.head(5)

,group_id,discipline_id,kind,student_id,state,period,semester
4855,621,20,lecture,12347.0,wasnt,1.0,0.0
4856,621,20,lecture,12348.0,wasnt,1.0,0.0
4857,621,20,lecture,11807.0,wasnt,1.0,0.0
4858,621,20,lecture,11808.0,wasnt,1.0,0.0
4859,621,20,lecture,11809.0,wasnt,1.0,0.0


In [625]:
buffer_table.loc[buffer_table['state'] == 'was', 'state'] = 1
buffer_table.loc[buffer_table['state'] == 'wasnt', 'state'] = 0

buffer_table = buffer_table.reset_index()

buffer_table.drop('index', axis=1, inplace=True)

buffer_table.head(5)

,group_id,discipline_id,kind,student_id,state,period,semester
0,621,20,lecture,12347.0,0,1.0,0.0
1,621,20,lecture,12348.0,0,1.0,0.0
2,621,20,lecture,11807.0,0,1.0,0.0
3,621,20,lecture,11808.0,0,1.0,0.0
4,621,20,lecture,11809.0,0,1.0,0.0


In [626]:
sum = buffer_table
sum = sum.groupby(['group_id', 'discipline_id', 'kind', 'student_id', 'period', 'semester'])['state'].sum().reset_index().rename(columns={'state':'count_presences'})

sum.head(5)

,group_id,discipline_id,kind,student_id,period,semester,count_presences
0,341,4,lecture,6412.0,2.0,0.0,0
1,341,4,lecture,6412.0,3.0,0.0,1
2,341,4,lecture,6414.0,2.0,0.0,1
3,341,4,lecture,6414.0,3.0,0.0,1
4,341,4,lecture,6424.0,2.0,0.0,0


In [627]:
count = buffer_table
count = count.groupby(['group_id', 'discipline_id', 'kind', 'student_id', 'period', 'semester'])['state'].count().reset_index().rename(columns={'state':'count_lessons'})

count.head(5)

,group_id,discipline_id,kind,student_id,period,semester,count_lessons
0,341,4,lecture,6412.0,2.0,0.0,2
1,341,4,lecture,6412.0,3.0,0.0,1
2,341,4,lecture,6414.0,2.0,0.0,2
3,341,4,lecture,6414.0,3.0,0.0,1
4,341,4,lecture,6424.0,2.0,0.0,2


In [628]:
buffer_table = sum.join(count[['count_lessons']]).dropna()

buffer_table.head(5)

,group_id,discipline_id,kind,student_id,period,semester,count_presences,count_lessons
0,341,4,lecture,6412.0,2.0,0.0,0,2
1,341,4,lecture,6412.0,3.0,0.0,1,1
2,341,4,lecture,6414.0,2.0,0.0,1,2
3,341,4,lecture,6414.0,3.0,0.0,1,1
4,341,4,lecture,6424.0,2.0,0.0,0,2


In [629]:
buffer_table['attendance'] = buffer_table['count_presences'] / buffer_table['count_lessons']

buffer_table.drop('count_presences', axis=1, inplace=True)
buffer_table.drop('count_lessons', axis=1, inplace=True)

buffer_table.head(5)

,group_id,discipline_id,kind,student_id,period,semester,attendance
0,341,4,lecture,6412.0,2.0,0.0,0.0
1,341,4,lecture,6412.0,3.0,0.0,1.0
2,341,4,lecture,6414.0,2.0,0.0,0.5
3,341,4,lecture,6414.0,3.0,0.0,1.0
4,341,4,lecture,6424.0,2.0,0.0,0.0


In [630]:
buffer_table = buffer_table.join(attendance_disciplines.set_index('id'), on='discipline_id').dropna()

buffer_table.drop('discipline_id', axis=1, inplace=True)
buffer_table.drop('group_id', axis=1, inplace=True)
buffer_table = buffer_table.rename(columns={'student_id':'attendance_id'})

buffer_table.head(5)

,kind,attendance_id,period,semester,attendance,title
0,lecture,6412.0,2.0,0.0,0.0,Правоведение
1,lecture,6412.0,3.0,0.0,1.0,Правоведение
2,lecture,6414.0,2.0,0.0,0.5,Правоведение
3,lecture,6414.0,3.0,0.0,1.0,Правоведение
4,lecture,6424.0,2.0,0.0,0.0,Правоведение


In [631]:
master_table = master_table.join(buffer_table.set_index(['attendance_id', 'semester']), on=['attendance_id', 'semester']).dropna()

master_table.head(5)

,con_id,attendance_id,ocenka_id,group_num,group_id_ocenka,group_id_attendance,group_id_timetable,semester,discipline_id_timetable,dicipline_name_timetable,kind,period,attendance,title
0,157460,11850,12100,438-3,482,623,6816.0,0,23733.0,История,lecture,1.0,1.0,Математика
0,157460,11850,12100,438-3,482,623,6816.0,0,23733.0,История,lecture,2.0,1.0,Математика
0,157460,11850,12100,438-3,482,623,6816.0,0,23733.0,История,lecture,3.0,0.5,Математика
0,157460,11850,12100,438-3,482,623,6816.0,0,23733.0,История,practice,1.0,0.777778,Математика
0,157460,11850,12100,438-3,482,623,6816.0,0,23733.0,История,practice,2.0,1.0,Математика


Собираем оценки

In [632]:
ocenka_semesters = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-semesters.csv").rename(columns={'id':'semester_id'})

ocenka_semesters.head()

,semester_id,academic_year,kind
0,11,2018/2019,autumn
1,12,2018/2019,spring
2,13,2019/2020,autumn
3,14,2019/2020,spring
4,15,2020/2021,autumn


In [633]:
ocenka_discipline_semesters = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-discipline-semesters.csv").rename(columns={'id':'discipline_semester_id'})

ocenka_discipline_semesters.head(5)

,discipline_semester_id,discipline_id,semester_id
0,127,127,1
1,128,128,1
2,129,129,1
3,130,130,1
4,11396,130,8


In [634]:
ocenka_disciplines = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-disciplines.csv").rename(columns={'id':'discipline_id'})

ocenka_disciplines.head(5)

,discipline_id,group_id,title,abbr
0,127,18,Математика,Математика
1,128,18,Математическая логика и теория алгоритмов,МЛиТА
2,129,18,Физика,Физика
3,130,18,Структуры и алгоритмы обработки данных в ЭВМ,СиАОДвЭВМ
4,131,18,ЭВМ и периферийные устройства,ЭВМиПУ


In [635]:
ocenka_marks = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-marks.csv").rename(columns={'id':'mark_id'})

ocenka_marks.head(5)

,mark_id,grade,student_id,discipline_semester_period_id,kind,marked_on,passed_on
0,766212,3,11162,43924,diff_qualification,2018-01-30,2018-01-16
1,780288,0,11162,43922,exam,2018-01-29,2018-01-12
2,780613,0,11162,43925,exam,2018-01-29,2018-01-16
3,780618,0,11162,43923,exam,2018-01-29,2018-01-23
4,784056,0,11162,43926,exam,2018-01-30,2018-01-30


In [636]:
ocenka_discipline_semesters_periods = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-discipline-semester-periods.csv").rename(columns={'id':'discipline_semester_period_id'})

ocenka_discipline_semesters_periods.head(5)

,discipline_semester_period_id,discipline_semester_id,period_id
0,127,127,2
1,24655,127,66
2,28513,127,67
3,128,128,2
4,24657,128,66


In [637]:
buffer_table = ocenka_semesters

buffer_table.head(5)

,semester_id,academic_year,kind
0,11,2018/2019,autumn
1,12,2018/2019,spring
2,13,2019/2020,autumn
3,14,2019/2020,spring
4,15,2020/2021,autumn


In [638]:
buffer_table.drop(['kind', 'academic_year'], axis=1, inplace=True)

buffer_table['semester_num'] = buffer_table['semester_id'] - 11

buffer_table.head(5)

,semester_id,semester_num
0,11,0
1,12,1
2,13,2
3,14,3
4,15,4


In [639]:
buffer_table = buffer_table.join(ocenka_discipline_semesters.set_index('semester_id'), on='semester_id').dropna()

buffer_table.head(5)

,semester_id,semester_num,discipline_semester_id,discipline_id
0,11,0,19150,12096
0,11,0,19233,12105
0,11,0,19355,13465
0,11,0,19543,13552
0,11,0,19149,15477


In [640]:
buffer_table = buffer_table.join(ocenka_disciplines.set_index('discipline_id'), on='discipline_id').drop('abbr', axis=1).dropna()

buffer_table.head(5)

,semester_id,semester_num,discipline_semester_id,discipline_id,group_id,title
0,11,0,19150,12096,413,Математика
0,11,0,19233,12105,414,Математика
0,11,0,19355,13465,363,Физика для информатики
0,11,0,19543,13552,374,Физика для информатики
0,11,0,19149,15477,413,Философия


In [641]:
buffer_table = buffer_table.join(ocenka_discipline_semesters_periods.set_index('discipline_semester_id'), on='discipline_semester_id').dropna()

buffer_table.head(5)

,semester_id,semester_num,discipline_semester_id,discipline_id,group_id,title,discipline_semester_period_id,period_id
0,11,0,19150,12096,413,Математика,52119.0,95.0
0,11,0,19150,12096,413,Математика,54879.0,97.0
0,11,0,19150,12096,413,Математика,58084.0,96.0
0,11,0,19233,12105,414,Математика,52127.0,95.0
0,11,0,19233,12105,414,Математика,54887.0,97.0


In [642]:
buffer_table = buffer_table.drop(['semester_id', 'discipline_semester_id', 'discipline_id', 'period_id', 'group_id'], axis=1).join(ocenka_marks.drop(['mark_id', 'marked_on', 'passed_on'], axis=1).set_index('discipline_semester_period_id'), on='discipline_semester_period_id').dropna().drop('discipline_semester_period_id', axis=1)

buffer_table.head(5)

,semester_num,title,grade,student_id,kind
0,0,Математика,4.0,11246.0,kt_1
0,0,Математика,4.0,11246.0,kt_2
0,0,Математика,4.0,11246.0,exam
0,0,Математика,0.0,14098.0,kt_2
0,0,Математика,0.0,14098.0,exam


In [643]:
buffer_table.rename(columns={'semester_num':'semester', 'title':'discipline'}, inplace=True)
buffer_table.reset_index(inplace=True)
buffer_table.drop('index', axis=1, inplace=True)

buffer_table.head(5)

,semester,discipline,grade,student_id,period
0,0,Математика,4.0,11246.0,kt_1
1,0,Математика,4.0,11246.0,kt_2
2,0,Математика,4.0,11246.0,exam
3,0,Математика,0.0,14098.0,kt_2
4,0,Математика,0.0,14098.0,exam


In [ ]:
buffer_table.loc[(buffer_table['kind'] == 'exam') | (buffer_table['kind'] == 'qualification') | (buffer_table['kind'] == 'diff_qualification') , 'period'] = 3
buffer_table.loc[(buffer_table['kind'] == 'kt_2'), 'period'] = 2
buffer_table.loc[(buffer_table['kind'] == 'kt_1'), 'period'] = 1

buffer_table.drop('kind', axis=1, inplase=True)